In [1]:
import numpy as np
import random as random
import simpleguitk as simplegui
import time

In [79]:
#display color with dictionary
dict_col={-2:'blue',-1:'blue',0:'cyan',1:'green',2:'green',3:'green',4:'green',5:'yellow',6:'yellow',7:'yellow',8:'yellow',9:'red',10:'red',11:'red',12:'red'}

In [80]:
class Card:
    global dict_col
    def __init__(self, number):
        self.number = number
        
        self.color = dict_col[number]
        #open default is closed
        self.open =False
        self.position=[None,None]
    def __str__(self):
        if self.open==False:
            return "Card has Value "+str(self.number)+" and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Card has Value "+str(self.number)+" and is open."+ \
                " Its position is at "+str(self.position)
        #set card to state
    def set_state(self,state):
        self.open=state
    def set_number(self,number):
        self.number=number
        self.color = dict_col[number]
    #set position    
    def set_position(self,posinput):
        self.position=posinput         
    def draw(self,canvas):
        #self.position=list(pos)
        card_size=[70,50]
        corners=[[self.position],[self.position[0]+card_size[0],self.position[1]],[self.position[0]+card_size[0],self.position[1]+card_size[1]],[self.position[0],self.position[1]+card_size[1]]]
        if self.open==False:
            canvas.draw_polygon(corners,1,'Black','Black')
        else:    
            canvas.draw_polygon(corners,1,'Black','Light Gray')
            centerb=list(self.position)
            centerb[1]+=58
            centerb[0]+=5
            canvas.draw_text(self.number,centerb,50,self.color)

Check the class functions below. Display still needs to be added. 

In [81]:
card1=Card(-2)
print(card1.color)
print(card1.open)
ar=np.zeros((2))
#number needs to be in array Crad does not work
#not sure whether Object Oriented is useful for speed, clearer for understandable
ar[0]=card1.number
ar[1]=card1.open
print(ar)
print(card1)
card1.set_state(True)
print(card1)
card1.set_state(False)
print(card1)

blue
False
[-2.  0.]
Card has Value -2 and is closed. Its position is at [None, None]
Card has Value -2 and is open. Its position is at [None, None]
Card has Value -2 and is closed. Its position is at [None, None]


Class which creates (lists) can be created from scratch or from another pile/set.

In [82]:
class Pile:
    def __init__(self,mode,take_from):
        #create new close pile, shuffles it also 
        if mode=='create_closed':
            #all closed when 
            self.open=False
            #postion in gui
            self.position=[100,10]
            #number of cards per type
            dict_num={-2:5,-1:10,0:15,1:10,2:10,3:10,4:10,5:10,6:10,7:10,8:10,9:10,10:10,11:10,12:10}
            self.list_cards=[]
            for i in range(15):
                for j in range(dict_num[i-2]):
                    cardin=Card(i-2)
                    cardin.set_position(self.position)
                    cardin.set_state(False)
                    self.list_cards.append(cardin)     
                    self.list_cards
            random.shuffle(self.list_cards)     
        #create open pile from other    
        if mode=='create_open': 
            #postion in gui
            self.position=[300,10]            
            self.list_cards=[]
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[0].set_state(True)
            self.open=True
            self.list_cards[0].set_position(self.position)            
    def __str__(self): 
        if self.open==False:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is closed."+ \
                " Its position is at "+str(self.position)
        if self.open==True:
            return "Pile has length of "+str(len(self.list_cards))+" Cards and is open."+ \
                " Its position is at "+str(self.position)
    #functions needed, recreate pile if nearly empty
    #giving (always top most card) and getting cards
    def give_card(self):
        card=self.list_cards.pop(-1)
        return card
    def get_card(self,card):
        self.list_cards.append(card)
        #set to position and state of pile
        self.list_cards[-1].set_position(self.position)
        self.list_cards[-1].set_state(self.open)

Create new pile and test that it looks right. 

In [83]:
pile_closed=Pile('create_closed',False)
print(pile_closed)
for i in range(len(pile_closed.list_cards)):
    print(pile_closed.list_cards[i])  

Pile has length of 150 Cards and is closed. Its position is at [100, 10]
Card has Value 6 and is closed. Its position is at [100, 10]
Card has Value -1 and is closed. Its position is at [100, 10]
Card has Value 4 and is closed. Its position is at [100, 10]
Card has Value 6 and is closed. Its position is at [100, 10]
Card has Value 11 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value 10 and is closed. Its position is at [100, 10]
Card has Value 9 and is closed. Its position is at [100, 10]
Card has Value 3 and is closed. Its position is at [100, 10]
Card has Value 5 and is closed. Its position is at [100, 10]
Card has Value 12 and is closed. Its position is at [100, 10]
Card has Value 3 and is closed. Its position is at [100, 10]
Card has Value 3 and is closed. Its position is at [100, 10]
Card has Value 0 and is closed. Its position is at [100, 10]
Card has Value 0 and is closed. Its position is at [100, 10]
Card ha

Now test make second open pile.

In [84]:
pile_open=Pile('create_open',pile_closed)
print(pile_open)
print(len(pile_open.list_cards),len(pile_closed.list_cards))

Pile has length of 1 Cards and is open. Its position is at [300, 10]
1 149


In [85]:
for i in range(len(pile_open.list_cards)):
    print(pile_open.list_cards[i]) 

Card has Value 9 and is open. Its position is at [300, 10]


In [86]:
#testing of giving and getting cards
card_a=pile_closed.give_card()
print(card_a)
print(len(pile_closed.list_cards))
print(pile_closed.list_cards[-1])
pile_open.get_card(card_a)
print(len(pile_open.list_cards),pile_open.list_cards[-1])
#works. But need to get to player also 

Card has Value 3 and is closed. Its position is at [100, 10]
148
Card has Value 3 and is closed. Its position is at [100, 10]
2 Card has Value 3 and is open. Its position is at [300, 10]


Now I create the players clas which has the cards. The cards are aranged in a 4 rows of 3 pattern. A Card Position can be empty. 

In [91]:
#input list of players
#random if random cards are opened, from which pile cards are taken
#for some purposes array better but here tried as list
class Player:
    global dict_col
    #parameters, name mode (human, computer), take_from pile, level (number for mode, more orptions for computer then human) 
    def __init__(self,name,mode,take_from, level):
        #grid positions 
        self.positionx=[0,70,140,210,0,70,140,210,0,70,140,210]
        self.positiony=[0,0,0,0,50,50,50,50,100,100,100,100]
        #whether cards exist
        self.exist=[]
        self.list_cards=[]
        self.name=name
        #check mode computer (different levels)/human
        if mode=='computer':
            self.mode='computer'
        if mode=='human':
            self.mode='human'
        #level how random     
        self.level=level    
 
        for i in range(12):
            self.exist.append(1)
            self.list_cards.append(take_from.list_cards.pop(-1))
            self.list_cards[-1].set_state(False)
            pos=list([self.positionx[i],self.positiony[i]])
            self.list_cards[-1].set_position(pos) 
        #list for shuffling and set two to open , currently only randomly implemented
        if level<10:
            rang=list(range(12))
            random.shuffle(rang)
            self.list_cards[rang[0]].set_state(True)
            self.list_cards[rang[1]].set_state(True)
        #option for player to do that interactive to be added     
    def __str__(self):
        return "Player "+self.name+" is a "+self.mode+" in level "+str(self.level)+"."
    def give_card(self,i):
        card=self.list_cards[i]
        self.exist[i]=0
        self.list_cards[i]=None
        return card
    #is always open then is at certain position changed 
    def get_card(self,card,i):
        self.list_cards[i]=card
        #set to position and to open and to existing
        self.exist[i]=1
        pos=list([self.positionx[i],self.positiony[i]])
        self.list_cards[i].set_position(pos)
        self.list_cards[i].set_state(True)
    #function only for testing    
    #card, id of card in list, number, state     
    def set_value(self,i,number,state):
        #set to desired value and position
        self.exist[i]=1  
        self.list_cards[i].set_state(True)     
        self.list_cards[i].set_number(number)        
    #gets list of closed cards    
    def get_all_closed(self):   
        closed=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==False:
                    closed.append(j)
        print(self.name+" has "+str(len(closed))+" closed cards")            
        return closed     
    #gets list of open cards      
    def get_all_open(self):   
        open2=[]
        for j in range(12):
            if self.exist[j]==1:
                if self.list_cards[j].open==True:
                    open2.append(j)
        print(self.name+" has "+str(len(open2))+" open cards")            
        return open2 
    #get list of existing cards
    def get_all_cards(self):   
        existing=[]
        for j in range(12):
            if self.exist[j]==1:
                existing.append(j)
        print(self.name+" has "+str(len(existing))+" cards")            
        return existing    
    #get score at the end
    def get_score(self):   
        score=0
        for j in range(12):
            if self.exist[j]==1:
                score+=self.list_cards[j].number         
        return score 
    #check whether cards vanishes
    def check_vanish_cards(self):
        vanish=-1
        for j in range(3):
            #3 open of same number
            if self.exist[0+j]==True and self.exist[4+j]==True and self.exist[8+j]==True: 
                if self.list_cards[0+j].open==True and self.list_cards[4+j].open==True and self.list_cards[8+j].open==True and self.list_cards[0+j].number==self.list_cards[4+j].number and self.list_cards[0+j].number==self.list_cards[8+j].number:
                    vanish=j
        return vanish  
    #remove when they are marked vanish
    def vanish_cards(self,row):
        card1=self.list_cards[0+row]
        card2=self.list_cards[4+row]
        card3=self.list_cards[8+row]
        self.exist[0+row]=0
        self.exist[4+row]=0
        self.exist[8+row]=0
        self.list_cards[0+row]=None
        self.list_cards[4+row]=None
        self.list_cards[8+row]=None
        return card1, card2, card3              

Test code below.

In [92]:
#recreate piles to start from normal length
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 
print(alpha,beta)
closed=alpha.get_all_closed()
open2=alpha.get_all_open()
exist=beta.get_all_cards()
exist2=alpha.get_all_cards()
score=alpha.get_score()
print(score)
#checking vanishing 
check=alpha.check_vanish_cards()
print(check)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
alpha.set_value(0,5,True)
alpha.set_value(4,5,True)
alpha.set_value(8,5,True)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
check=alpha.check_vanish_cards()
print(check)
card1,card2,card3=alpha.vanish_cards(check)
print(alpha.list_cards[0])
print(alpha.list_cards[4])
print(alpha.list_cards[8])
print(card1,card2,card3)

Player alpha is a computer in level 0. Player beta is a computer in level 0.
alpha has 10 closed cards
alpha has 2 open cards
beta has 12 cards
alpha has 12 cards
72
-1
Card has Value 7 and is closed. Its position is at [0, 0]
Card has Value 9 and is closed. Its position is at [0, 50]
Card has Value 9 and is closed. Its position is at [0, 100]
Card has Value 5 and is open. Its position is at [0, 0]
Card has Value 5 and is open. Its position is at [0, 50]
Card has Value 5 and is open. Its position is at [0, 100]
0
None
None
None
Card has Value 5 and is open. Its position is at [0, 0] Card has Value 5 and is open. Its position is at [0, 50] Card has Value 5 and is open. Its position is at [0, 100]


In [44]:
for i in range(12):
    print(i)
    print(exist[i])
    print(alpha.list_cards[i])

0
0
Card has Value 12 and is open. Its position is at [0, 0]
1
1
Card has Value 5 and is closed. Its position is at [70, 0]
2
2
Card has Value 0 and is closed. Its position is at [140, 0]
3
3
Card has Value 8 and is closed. Its position is at [210, 0]
4
4
Card has Value 5 and is closed. Its position is at [0, 50]
5
5
Card has Value 4 and is closed. Its position is at [70, 50]
6
6
Card has Value 0 and is closed. Its position is at [140, 50]
7
7
Card has Value 9 and is closed. Its position is at [210, 50]
8
8
Card has Value 1 and is open. Its position is at [0, 100]
9
9
Card has Value 2 and is closed. Its position is at [70, 100]
10
10
Card has Value -1 and is closed. Its position is at [140, 100]
11
11
Card has Value 8 and is closed. Its position is at [210, 100]


In [11]:
for i in range(len(alpha.list_cards)):
    print(f" {i} {alpha.list_cards[i]}")  

 0 Card has Value 0 and is open. Its position is at [0, 0]
 1 Card has Value 0 and is closed. Its position is at [70, 0]
 2 Card has Value 12 and is closed. Its position is at [140, 0]
 3 Card has Value 6 and is closed. Its position is at [210, 0]
 4 Card has Value 0 and is closed. Its position is at [0, 50]
 5 Card has Value 0 and is closed. Its position is at [70, 50]
 6 Card has Value 7 and is closed. Its position is at [140, 50]
 7 Card has Value 3 and is closed. Its position is at [210, 50]
 8 Card has Value 4 and is closed. Its position is at [0, 100]
 9 Card has Value 2 and is open. Its position is at [70, 100]
 10 Card has Value 11 and is closed. Its position is at [140, 100]
 11 Card has Value 9 and is closed. Its position is at [210, 100]


In [12]:
#take giving and taking cards
print(f" {1} {alpha.list_cards[1]}")
card_a=alpha.give_card(1)
print(card_a)
print(f" {1} {alpha.list_cards[1]}")
print(pile_open.list_cards[-1])      
pile_open.get_card(card_a)      
print(pile_open.list_cards[-1])  

 1 Card has Value 0 and is closed. Its position is at [70, 0]
Card has Value 0 and is closed. Its position is at [70, 0]
 1 None
Card has Value 8 and is open. Its position is at [300, 10]
Card has Value 0 and is open. Its position is at [300, 10]


In [13]:
print(pile_closed.list_cards[-1])
card_b=pile_closed.give_card()
alpha.get_card(card_b,1)
print(f" {1} {alpha.list_cards[1]}") 
print(len(pile_closed.list_cards))    
print(pile_closed.list_cards[-1]) 

Card has Value 11 and is closed. Its position is at [100, 10]
 1 Card has Value 11 and is open. Its position is at [70, 0]
124
Card has Value 6 and is closed. Its position is at [100, 10]


Exchanging cards (vanishing also means only one step fore a player).

Create all again to start from start state.

In [14]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 

Create list of players. Vary number for testing different cases.

In [15]:
#different test cases
#players=[alpha,beta,gamma,delta]
#players=[alpha,beta,gamma]
players=[alpha,beta]
#players=[alpha,beta,gamma,delta,epsilon,zeta,eta,theta]
#need better print function
print(players)
for i in range(len(players)):
    print(players[i])

[<__main__.Player object at 0x7fd33ff10280>, <__main__.Player object at 0x7fd314199af0>]
Player alpha is a computer in level 0.
Player beta is a computer in level 0.


Function to determine which player starts.

In [16]:
#parameters, whether is first round of game, player_list, last_ended (which means last action of previous game)
def who_starts(first,players,last_ender):
    #if first round of a game largest one starts
    if first==True:
        #first is default 
        starter=players[0]
        for i in range(len(players)):
            counter=0
            for j in range(12):
                #when card exists
                if players[i].exist[j]==1:
                    #if card open
                    if players[i].list_cards[j].open==1:
                        counter+=players[i].list_cards[j].number
            if i==0:
                lowcounter=counter
            if i>0 and counter>lowcounter:
                starter=players[i]
        print("Player "+str(starter.name)+" starts")        
        return starter  
    else:
        print("Player "+str(last_ender.name)+" starts") 
        return last_ender                    

In [17]:
#test function
a=who_starts(True,players,None)
b=who_starts(False,players,beta)

Player alpha starts
Player beta starts


Function to make a turn, first computer option. Needs later different choices for levels. 

In [98]:
#players is only needed that higher level computer can consider other players card for choices
#possible add past scores for that purpose also
def turn(player,players,pile,discarded):
    #global score(needed when several rounds) and inplay
    global score, in_play
    if player.mode=='computer':
        print('mode is computer level '+str(player.level))
        #in level 0 random 50% choice of action
        if player.level==0:
            r_number1=random.random()
            print(r_number1)
            if r_number1<0.5:
                take_open=False
            else:    
                take_open=True
        if take_open==True:
            #by design
            discard=False
            print("take from open swaps with own")
            #use method for existing cards
            existing=player.get_all_cards()
            #select from them
            #randomly for level 0
            if player.level==0:
                selected=random.choice(existing)
            card_a=player.give_card(selected)
            card_b=pile_open.give_card()
            pile_open.get_card(card_a)
            player.get_card(card_b,selected)
            print("Player has now "+str(player.list_cards[selected].number))
            print("open Pile has now on "+str(pile_open.list_cards[-1].number))            
        else:
            #in level 0 random 50% choice of sub action
            if player.level==0:
                r_number2=random.random()
                print(r_number2)
                if r_number2<0.5:
                    discard=False
                else:    
                    discard=True
            if discard==True:
                print("take from closed discards it")
                #put card on open pile from closed
                card_a=pile_closed.give_card()
                pile_open.get_card(card_a)
                #open random closed cards using method
                closed=player.get_all_closed()
                #randomly for level 0
                if player.level==0:
                    selected=random.choice(closed)  
                player.list_cards[selected].set_state(True)
                print("Player opened "+str(player.list_cards[selected].number))
                print("open Pile has now on top "+str(pile_open.list_cards[-1].number))                
            #next close card not discard mode (last mode)  
            else:
                print("take from closed swaps with own")
                #choose from existing cards
                existing=player.get_all_cards()
                #randomly for level 0
                if player.level==0:
                    selected=random.choice(existing)
                card_a=player.give_card(selected)
                card_b=pile_closed.give_card()
                pile_open.get_card(card_a)
                player.get_card(card_b,selected)
                print("Player has now "+str(player.list_cards[selected].number))
                print("open Pile has now in top "+str(pile_open.list_cards[-1].number))
    #check for vanishing cards
    van=player.check_vanish_cards()
    if van>-1:
        card1,card2,card3=player.vanish_cards(van)
        pile_open.get_card(card1)
        pile_open.get_card(card2)
        pile_open.get_card(card3)
        print("3 "+str(card1.number)+" vanish from Player")
    #check whether there are still closed cards 
    closed=player.get_all_closed()
    if len(closed)==0:
        in_play=False 
        print("player "+player.name+" opened all cards")
    return player, players, pile, discarded  

Playing some turns, it seems to work. 

In [99]:
pile_closed=Pile('create_closed',False)
pile_open=Pile('create_open',pile_closed)
alpha=Player("alpha",'computer',pile_closed,0)
beta=Player("beta",'computer',pile_closed,0) 
players=[alpha,beta]
starter=who_starts(True,players,None)
turn(alpha,players,pile_closed,pile_open)
print(len(alpha.get_all_closed()))

Player beta starts
mode is computer level 0
0.35385749845402026
0.6827924685595955
take from closed discards it
alpha has 10 closed cards
Player opened 6
open Pile has now on top 12
alpha has 9 closed cards
alpha has 9 closed cards
9


Now creating function for round. 

In [100]:
#parameters player names list, mode list, level, list, pause per turn(?)  gui anyway separated yet 
#but will need later, also regarding collecting output
def skyjo_round(names,nature,levels,pause):
    global in_play, players, pile_open, pile_closed
    pile_closed=Pile('create_closed',False)
    print(len(pile_closed.list_cards))
    pile_open=Pile('create_open',pile_closed)
    players=[]
    for i in range(len(names)):
        players.append(Player(names[i],nature[i],pile_closed,levels[i]))
    #determine start player, for now it assumes, is a first round    
    starter=who_starts(True,players,None)
    print(starter)
    #index of starter
    idx=players.index(starter)
    print(idx)
    if pause>0:
        time.sleep(pause)
    #first turn
    turn(starter,players,pile_closed,pile_open)
    #counter set to index of starter
    counter=idx
    in_play=True
    while in_play==True:
        if pause>0:
            time.sleep(pause)
        #now alternating turn while in play
        counter+=1
        player=players[counter%len(players)]
        print("next turn is of "+player.name)
        player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open)
        print(len(pile_closed.list_cards))
        
    #one turn of the other players at the  end
    if in_play==False:
        for i in range(len(players)-1):
            if pause>0:
                time.sleep(pause)
            counter+=1
            player=players[counter%len(players)]
            print("next turn is of "+player.name)
            player,players,pile_closed,pile_open=turn(player,players,pile_closed,pile_open)
    scores=[]
    print(len(players))
    for i in range(len(players)):
        scores.append(players[i].get_score())              
    return scores, counter-idx, players

In [101]:
#test round # seems to works but less clear why it works and why the number of globals and returns are needed
#creates names and properties of player
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
nn=int(200)
start_time=time.time()
result=np.zeros((4,nn))
for i in range(nn):
    scores,turns,players=skyjo_round(names,nature,levels,0)
    print(scores)
    print(turns)  
    result[0,i]=scores[0]
    result[1,i]=scores[1]
    result[2,i]=turns
stop_time=time.time()
print(stop_time-start_time)
print(result)

150
Player beta starts
Player beta is a computer in level 0.
1
mode is computer level 0
0.6751373506387556
take from open swaps with own
beta has 12 cards
Player has now 5
open Pile has now on 10
beta has 9 closed cards
next turn is of alpha
mode is computer level 0
0.9975823249476373
take from open swaps with own
alpha has 12 cards
Player has now 10
open Pile has now on 11
alpha has 9 closed cards
125
next turn is of beta
mode is computer level 0
0.5238054608092053
take from open swaps with own
beta has 12 cards
Player has now 11
open Pile has now on 1
beta has 8 closed cards
125
next turn is of alpha
mode is computer level 0
0.7863787852099945
take from open swaps with own
alpha has 12 cards
Player has now 1
open Pile has now on 12
alpha has 8 closed cards
125
next turn is of beta
mode is computer level 0
0.4280419153298246
0.7093867289642785
take from closed discards it
beta has 8 closed cards
Player opened 10
open Pile has now on top 1
beta has 7 closed cards
124
next turn is of al

alpha has 1 closed cards
111
next turn is of beta
mode is computer level 0
0.4676458360993636
0.2360649928427594
take from closed swaps with own
beta has 12 cards
Player has now 3
open Pile has now in top 1
beta has 1 closed cards
110
next turn is of alpha
mode is computer level 0
0.1169912801722579
0.5452809997968465
take from closed discards it
alpha has 1 closed cards
Player opened 10
open Pile has now on top -2
alpha has 0 closed cards
player alpha opened all cards
109
next turn is of beta
mode is computer level 0
0.5881287446722063
take from open swaps with own
beta has 12 cards
Player has now -2
open Pile has now on 11
beta has 1 closed cards
2
[80, 67]
39
150
Player alpha starts
Player alpha is a computer in level 0.
0
mode is computer level 0
0.6105908452344594
take from open swaps with own
alpha has 12 cards
Player has now 4
open Pile has now on 6
alpha has 9 closed cards
next turn is of beta
mode is computer level 0
0.8615623665325889
take from open swaps with own
beta has 12

Function to draw the results in the GUI  screen.  Should be disactivated for computer only.

In [102]:
# draw function
def draw(canvas):
    global pile_open,pile_closed, players
    #display the top most card
    p_open=pile_open.list_cards[-1]
    p_closed=pile_closed.list_cards[-1]
    # test to make sure that card.draw works
    p_open.draw(canvas)
    p_closed.draw(canvas)
    for i in range(len(players)):
        for j in range(12):
            #only cards which exist are drawn:
            if players[i].exist[j]==1:
                card=players[i].list_cards[j]
                if len(players)==2:
                    drawpos=list(card.position)
                    drawpos[0]=i*290+players[i].positionx[j]
                    drawpos[1]=100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-3.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%2)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//2)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%2)*290,180*(1+(i//2))+90),15,'Black')
                if abs(len(players)-5.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%3)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//3)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%3)*290,180*(1+(i//3))+90),15,'Black')     
                if abs(len(players)-7.5)==0.5:
                    drawpos=list(card.position)
                    drawpos[0]=(i%4)*290+players[i].positionx[j]
                    drawpos[1]=180*(i//4)+100+players[i].positiony[j]
                    card.set_position(drawpos)
                    card.draw(canvas)
                    canvas.draw_text(players[i].name,(100+(i%4)*290,180*(1+(i//4))+90),15,'Black')    

Drawing works now for all allowed player numbers. 

In [108]:
def hit_start():
    global start
    #better but still not really
    players,scores,turns=skyjo_round(names,nature,levels,1) 
    #only appears at the end

In [ ]:
#seems not work well in the notebook sometimes, not clear whether real problem
#likely ignored since notebook is not so important here
names=['alpha','beta']
nature=['computer','computer']
levels=[0,0]
if len(names)==2:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50+30) 
if abs(len(names)-3.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(1), 100+3*50*2+60)
if abs(len(names)-5.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(2)+10, 100+3*50*2+60)    
if abs(len(names)-7.5)==0.5:
    frame = simplegui.create_frame("Skyjo", 290+280*(3)+20, 100+3*50*2+60)  
#parmeter for being able to observe computer on screen it still does mnot work
start=False
frame.set_canvas_background("White")
frame.add_button("start", hit_start, 200)
#players,scores,turns=skyjo_round(names,nature,levels,0)  
frame.set_draw_handler(draw)
frame.start()    
#drawing of comuter does not work 

invalid command name "140544977973184_draw_handler"
    while executing
"140544977973184_draw_handler"
    ("after" script)
invalid command name "140543479963264_draw_handler"
    while executing
"140543479963264_draw_handler"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


150
Player alpha starts
Player alpha is a computer in level 0.
0
mode is computer level 0
0.815800084485755
take from open swaps with own
alpha has 12 cards
Player has now -2
open Pile has now on 5
alpha has 10 closed cards
next turn is of beta
mode is computer level 0
0.21126079925369712
0.32482060722714534
take from closed swaps with own
beta has 12 cards
Player has now 7
open Pile has now in top 11
beta has 9 closed cards
124
next turn is of alpha
mode is computer level 0
0.8787281967131624
take from open swaps with own
alpha has 12 cards
Player has now 11
open Pile has now on -2
alpha has 10 closed cards
124
next turn is of beta
mode is computer level 0
0.6444679453329092
take from open swaps with own
beta has 12 cards
Player has now -2
open Pile has now on 5
beta has 8 closed cards
124
next turn is of alpha
mode is computer level 0
0.6269290141779691
take from open swaps with own
alpha has 12 cards
Player has now 5
open Pile has now on 12
alpha has 9 closed cards
124
next turn is 

In [ ]:
exit()
